In [ ]:
import pandas as pd
import geopandas as gpd
import plotly.express as px

### Geojson format for Geographical Geometry and related information  

In [ ]:
countriesURL = "https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json"

In [ ]:
import requests
response = requests.get(countriesURL)
countries = response.json()
countries

In [ ]:
df_countries = pd.DataFrame(countries["features"]).head()

In [ ]:
df_gapminder = px.data.gapminder().query("year==2007").reset_index(drop=True)
df_gapminder.head()

In [ ]:
px.choropleth(
    df_gapminder,
    geojson=countriesURL,
    featureidkey='properties.name',
    locations='country',
    color='lifeExp',
    color_continuous_scale='Reds'
).update_geos(fitbounds="locations", visible=True, 
              lataxis_showgrid=True, lonaxis_showgrid=True, 
               #projection_type="mercator"
             ).update_layout(margin={"r":0,"t":0,"l":0,"b":0},
               #coloraxis=dict(colorbar=dict(orientation='h', y=1, len=0.5)))
                coloraxis=dict(colorbar=dict(x = 0.8,len=0.9)))

In [ ]:
px.choropleth(
    df_gapminder,
    geojson=countriesURL,
    featureidkey='id',
    locations='iso_alpha',
    color='lifeExp',
    color_continuous_scale='Reds'
).update_geos(fitbounds="locations", visible=True, 
              lataxis_showgrid=True, lonaxis_showgrid=True, 
             ).update_layout(margin={"r":0,"t":0,"l":0,"b":0},
               #coloraxis=dict(colorbar=dict(orientation='h', y=1, len=0.5)))
                coloraxis=dict(colorbar=dict(x = 0.8,len=0.9)))

In [ ]:
projections =  [ "albers" ,  "azimuthal equal area" , "conic equal area" , "conic equidistant" , "cylindrical equal area" , "equirectangular" , "mercator" , "natural earth" , "orthographic" , "robinson" , "stereographic" , "times" , "transverse mercator" , "winkel tripel"]

In [ ]:
fig=px.choropleth(
    df_gapminder,
    geojson=countriesURL,
    featureidkey='id',
    locations='iso_alpha',
    color='lifeExp',
    color_continuous_scale='Reds'
)
for projection in projections:
    
    fig.update_geos(projection_type=projection).update_layout(margin={"r":0,"l":0,"b":0},title={"text":"Projection used: "+projection})
               #coloraxis=dict(colorbar=dict(orientation='h', y=1, len=0.5)))
               # coloraxis=dict(colorbar=dict(x = 0.8,len=0.9)))
    fig.show()

### Bubble map

In [ ]:
df_LatLonPop = pd.read_csv("https://gist.githubusercontent.com/ofou/df09a6834a8421b4f376c875194915c9/raw/355eb56e164ddc3cd1a9467c524422cb674e71a9/country-capital-lat-long-population.csv")
df_LatLonPop.head()

In [ ]:
fig = px.scatter_geo(df_LatLonPop, 
        lon = "Longitude",
        lat = "Latitude",
        size="Population",
        hover_name="Capital City",
        title = "Population of World Capitals"
        #projection="natural earth"
).update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

### Mapbox (raster data)

In [ ]:
px.scatter_mapbox(df_LatLonPop,
        lon = "Longitude",
        lat = "Latitude",
        size="Population",
        hover_name="Capital City",
        title = "Population of World Capitals",
        mapbox_style="carto-positron",
        zoom=1, 
        center = {"lat": 20.5937, "lon": 78.9629},
        opacity=0.55,
).update_layout(margin={"r":0,"l":0,"b":0})

In [ ]:
fig=px.scatter_mapbox(df_LatLonPop,
        lon = "Longitude",
        lat = "Latitude",
        size="Population",
        hover_name="Capital City",
        title = "Population of World Capitals",
        mapbox_style="carto-positron",
        color_discrete_sequence=["fuchsia"],
        zoom=1, 
        center = {"lat": 20.5937, "lon": 78.9629},
        opacity=0.55,
)
fig.update_layout(margin={"r":0,"l":0,"b":0})
for style in ["open-street-map", "carto-positron", "carto-darkmatter"]:
    fig.update_layout(#mapbox_style=style,
        title={"text":"Mapbox style used:"+style},
        mapbox={
            "style": style,
            #"accesstoken":mapbox_access_token
        }
    )
    fig.show()

In [ ]:
px.choropleth_mapbox(
    df_gapminder,
    geojson=countriesURL,
    featureidkey='id',
    locations='iso_alpha',
    color='pop',
    mapbox_style="carto-positron",
    zoom=0, 
    opacity=0.5,
    color_continuous_scale='Reds', width=800, height = 500
)
#.update_layout(margin={"r":0,"t":0,"l":0,"b":0},
               #coloraxis=dict(colorbar=dict(orientation='h', y=1, len=0.5)))

## Using GeoPandas

GoePandas prefers geographical geometry data in binary shapefile (.shp) format. 
For current state, district boundaries you must download shape file from https://onlinemaps.surveyofindia.gov.in/
It uses Uses Lambert Conic Conformal mapping.

See https://en.wikipedia.org/wiki/Lambert_conformal_conic_projection

In [ ]:
indiaShapeFileURL = "https://github.com/datameet/maps/raw/master/Survey-of-India-Index-Maps/Boundaries/India-States.shp"
gdf_india = gpd.read_file(indiaShapeFileURL)
gdf_india.head()

In [ ]:
gdf_india.crs

In [ ]:
gdf_india.plot()

In [ ]:
gdf_india.boundary.plot()

In [ ]:
temp=gdf_india.copy()
temp=temp.to_crs(crs=24378)
print(temp.crs)

In [ ]:
temp1.plot(column="shape_area", legend="True").set_axis_off()

In [ ]:
temp1.plot(column="shape_area", legend="True", cmap='OrRd', legend_kwds={"label": "Shape Area"},  missing_kwds={
        "color": "lightgrey",
        "edgecolor": "red",
        "hatch": "///",
        "label": "Missing values",
    }).set_axis_off()

In [ ]:
centroids = temp.copy()
centroids.geometry = temp.centroid
centroids['size'] = centroids['shape_area'] / 1e9  # to get reasonable plotable number

ax = temp1.plot(facecolor='w', edgecolor='k')
centroids.plot(markersize='size', ax=ax)

In [ ]:
centroids.head()

In [ ]:
indiamap_json = gdf_india.to_json()

In [ ]:
indiamap_json

In [ ]:
gdf_india.to_file("output.geo.json", driver="GeoJSON")